# Data cleaning

先通过stata生成变量amount，num_goods, price_goods,读入数据，检查基本信息。
中国政府采购、公共采购主要分为货物、服务、工程三大类，有不同的公开招标金额标准。各省不同年份的标准不同，其中货物和服务的标准相同，工程按内容不同有不同标准。鉴于根据公开的采购信息，难以逐一判断工程类合同的具体内容，选择通过关键词判断合同是否为工程相关，直接排除工程部分，保留货物和服务采购数据。
下面的程序主要进行以下数据清洗：
1. 提取"项目名称","采购人甲方","采购人地址"中的省市县信息，结合2020年中国省市县三级行政区划表，匹配采购数据的地区(省级)
2. 根据"主要标的名称"和"项目名称"中词汇判断采购类型。
   出现的词频，选出top 100关键词，对关键词进行手动分类，排除其中容易出现歧义的部分，比如油，可能是“燃油采购”（货物），也可能是“加油服务”（服务）。对包含关键词的采购项目进行分类，在通过机器学习进行分类。

中国政府采购分为分散采购和集中采购两种，在大部分省份，当采购金额低于50万元时由地方分散采购，所以在我们的数据中，保留了50万元以上的采购项目。各省集中采购公开招标的门槛最大为400万元，以100万元为基础的带宽，保留数据到500万元。

In [30]:
import pandas as pd

pd.set_option("display.max_columns", None)   
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", None)     

In [ ]:

csv_file = "/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_clean1.csv"

df = pd.read_csv(csv_file, low_memory=False)

df_filtered = df[(df['amount'] >= 50) & (df['amount'] <= 500) & (df['年份'] >= 2020)]

In [ ]:
df_filtered['年份'].value_counts()

In [ ]:
df_filtered['amount'].describe()

## 确定省份

In [21]:
import geopandas as gpd

shp_path = "/Users/yxy/UChi/Summer2025/Procurement/raw/Countylevel_Admin_2020/China2020County.shp"
gdf = gpd.read_file(shp_path)

gdf = gdf[['省级', '地级', '县级']]

In [20]:
import re
def extract_prov_city_county(text: str):
    if not isinstance(text, str):  
        return None, None, None
    text = re.sub(r"[-_\s·、，,\.]()（）*。", "", text) 
    # text = re.sub(r"[^\u4e00-\u9fa5]", "", text)

    prov_pattern = r"(.*?(省|自治区|市))"    
    city_pattern = r"(.*?(市|地区|盟|州))"  
    county_pattern = r"(.*?(县|区|旗))"     
    
    prov, city, county = None, None, None
    municipalities = ["北京市","天津市","上海市","重庆市"]

    prov_match = re.match(prov_pattern, text)
    if prov_match:
        if '市' in prov_match.group(1) and prov_match.group(1) not in municipalities:
            prov = None
        else: 
            prov = prov_match.group(1)
            text = text[len(prov):]
    city_match = re.match(city_pattern, text)
    if city_match:
        city_candidate = city_match.group(1)
        if city_candidate.endswith("州") and text.startswith("市", len(city_candidate)):
            city = city_candidate + "市"
            text = text[len(city_candidate) + 1:] 
        else:
            city = city_candidate
            text = text[len(city):]
        
    county_match = re.match(county_pattern, text)
    if county_match:
        county = county_match.group(1)

    return prov, city, county


In [22]:

def fill_location(row):
    prov, city, county = None, None, None

    p1, c1, ct1 = extract_prov_city_county(row["采购人地址"])
    prov, city, county = p1, c1, ct1

    if prov is None or city is None or county is None:
        p2, c2, ct2 = extract_prov_city_county(row["采购人甲方"])
        if prov is None: prov = p2
        if city is None: city = c2
        if county is None: county = ct2

    if prov is None or city is None or county is None:
        p3, c3, ct3 = extract_prov_city_county(row["项目名称"])
        if prov is None: prov = p3
        if city is None: city = c3
        if county is None: county = ct3

    return pd.Series([prov, city, county])


In [23]:
def match_region(row, gdf):
    if row["prov"]:
        match = gdf[gdf["省级"] == row["prov"]]
        if not match.empty:
            return match["省级"].iloc[0]
    if row["city"]:
        match = gdf[gdf["地级"].str.contains(str(row["city"]).replace("市",""), na=False, regex=False)]
        if not match.empty:
            return match["省级"].iloc[0]
        match = gdf[gdf["县级"].str.contains(str(row["city"]).replace("市",""), na=False, regex=False)]
        if not match.empty:
            return match["省级"].iloc[0]
    if row["county"]:
        match = gdf[gdf["县级"] == row["county"]]
        if not match.empty:
            return match["省级"].iloc[0]

    return None


In [ ]:
df_filtered[["prov", "city", "county"]] = df_filtered.apply(fill_location, axis=1)
df_filtered["region"] = df_filtered.apply(lambda x: match_region(x, gdf), axis=1)
df_filtered.to_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_region.csv", index=False, encoding="utf-8-sig")

In [113]:
df_filtered = pd.read_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_region.csv", low_memory=False)

In [108]:
import jieba

def match_region_by_jieba(row, gdf):
    fields = ["采购人地址", "采购人甲方", "项目名称"]

    for field in fields:
        text = row.get(field, "")
        if not isinstance(text, str) or text.strip() == "":
            continue

        words = jieba.lcut(text)
        if not words:
            continue
        first_word = words[0]

        match = gdf[gdf["省级"].str.contains(first_word, na=False, regex=False)]
        if not match.empty:
            return match["省级"].iloc[0]

    return None

In [ ]:
mask = df_filtered["region"].isna()
df_filtered.loc[mask, "region"] = df_filtered[mask].apply(
    lambda x: match_region_by_jieba(x, gdf), axis=1
)
df_filtered = df_filtered[
    ~df_filtered["region"].isin(["澳门特别行政区", "台湾省","新疆维吾尔自治区","西藏自治区"])
]
df_filtered.to_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_region2.csv", index=False, encoding="utf-8-sig")

In [117]:
df_filtered = pd.read_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_region.csv", low_memory=False)
df_filtered2 = pd.read_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_region2.csv", low_memory=False)

## 确定类型
### top 200 items
给出现频率前200的标的物名称手动标注类别，使用了chatgpt+人工检查，对一部分“无”，“详情见合同”，标注了类别 “无分类”

In [120]:
out_path_kw = "/Users/yxy/UChi/Summer2025/Procurement/dta/keywords.csv"

In [188]:
df_filtered = pd.read_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_region.csv", low_memory=False)

In [189]:
df_filtered = df_filtered.dropna(subset=["region"])

In [190]:
df_filtered['年份'].value_counts(dropna=False)

年份
2023    191582
2022    158495
2021    105770
2020     47632
2024     24300
Name: count, dtype: int64

In [50]:
import pandas as pd
out_path_kw = "/Users/yxy/UChi/Summer2025/Procurement/dta/keywords.csv"
top100_items = df_filtered["主要标的名称"].value_counts().head(200).reset_index()
top100_items.columns = ["keyword", "count"]

top100_items["category"] = ""

top100_items.to_csv(out_path_kw, index=False, encoding="utf-8-sig")

print("Top 200 items exported to keywords.csv for manual categorization.")

Top 200 items exported to keywords.csv for manual categorization.


In [191]:
classified = pd.read_csv(out_path_kw)
df_filtered = df_filtered.merge(classified[["keyword", "category"]], 
              left_on="主要标的名称", 
              right_on="keyword", 
              how="left")
df_filtered.rename(columns={"category": "cat"}, inplace=True)
df_filtered.drop(columns=["keyword"], inplace=True)

In [197]:
df_filtered.loc[df_filtered["cat"].isna(), "主要标的名称"].value_counts(dropna=False)


主要标的名称
详见附件                    459
详见合同                    292
详见合同文本。                  97
详见招标文件                   73
/                        63
                       ... 
2023年清新区自然保护地整合优化项目       1
仙塘镇中心幼儿园基础设施配套补充项目        1
城区围挡更换仿真绿植                1
2023年鄂尔多斯全市旅游宣传片拍摄项目      1
电动手术床（电动液压手术台）            1
Name: count, Length: 24075, dtype: int64

In [60]:
import pandas as pd

# 读入两个csv
df1 = pd.read_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/keywords.csv")   
df2 = pd.read_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/cat_list.csv")  

df1 = df1[["keyword", "category"]]
df2 = df2[["keyword", "category"]]

df_all = pd.concat([df1, df2], ignore_index=True).drop_duplicates(subset=["keyword"])

df_all.to_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/keywords_with_cat.csv", index=False)


In [193]:
import pandas as pd
import jieba
import re
import numpy as np


kw_df = pd.read_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/keywords_with_cat.csv")

def clean_text(text):
    if not isinstance(text, str):
        return ""
    return re.sub(r"[^\u4e00-\u9fa50-9]", "", text)

def match_cat_from_text(text, kw_df):
    kw_df = kw_df.rename(columns={"category": "cat"})
    text = clean_text(text)
    if not text:
        return None
    words = jieba.lcut(text)
    for w in words:
        if w == "项目":
            continue
        match = kw_df[kw_df["keyword"].str.contains(w, na=False)]
        if not match.empty:
            valid = match[match["cat"] != "无分类"]
            if not valid.empty:
                return valid["cat"].iloc[0]
    return None


def assign_cat(row, kw_df):
    for col in ["主要标的名称", "项目名称"]:
        text = row.get(col, "")
        cat = match_cat_from_text(text, kw_df)
        if cat is not None:
            return cat
    return None



In [194]:
mask = df_filtered["cat"].isna() | (df_filtered["cat"] == "无分类")

df_filtered.loc[mask, "cat"] = df_filtered.loc[mask].apply(
    lambda x: assign_cat(x, kw_df), axis=1
)


In [195]:
df_filtered['cat'].value_counts(dropna=False)

cat
服务      223084
货物      187459
工程      100660
None     28657
Name: count, dtype: int64

In [ ]:
df_unmatched = df_filtered[df_filtered['cat'].isna()]
df_unmatched.sample(10)

## 采购方式

In [ ]:
methodlist = ["公开招标", "协议供货", "单一来源", "定点采购", 
    "电子卖场", "竞争性磋商", "竞争性谈判", "询价", "邀请招标"]

In [219]:
df_filtered = pd.read_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_primary.csv", low_memory=False)

In [233]:
methodlist = [
    "公开招标", "协议供货", "单一来源", "定点采购",
    "电子卖场", "竞争性磋商", "竞争性谈判", "询价", "邀请招标"
]

not_in_list = df_filtered.loc[~df_filtered["采购方式"].isin(methodlist)]

print("row count not in methodlist:", len(not_in_list))
print("ratio:", len(not_in_list) / len(df_filtered))

print(not_in_list["采购方式"].value_counts().head(100))


row count not in methodlist: 100093
ratio: 0.1854054754936465
采购方式
规格型号（或服务要求）服务要求包括图书出版年限、图书范围、与到货率、图书仓储及现场采购保障、到货验收、交货要求、时间要求及货源要求（具体要求详见合同）。                                                                         2
清华大学低温超导磁铁系统（更正）                                                                                                                                     1
主要标的名称采购计划编号 2023NCZ001016                                                                                                                           1
规格型号（或服务要求）根据和使用方的要求，完成阳江江城海事工作船码头后期竣工验收等项目。                                                                                                         1
Delta4 升级（带TOMO容积调强验证功能）（合同公告更正）                                                                                                                     1
主要标的名称本次项目通过公开确定一家供应商，服务期限1年，为采购人提供医院合群门诊、东川门诊、急诊、介入导管室、供应室、药学部、协和等医疗输送、勤杂服务。                                                                        1
主要标的名称我院放射科MRI和3台DR的保修期陆续于2

In [232]:
def clean_method(method: str, methodlist):
    if not isinstance(method, str):
        return None
    for m in methodlist:
        if m in method:  
            return m
    return method  

methodlist = ["公开招标", "协议供货", "单一来源", "定点采购",
              "电子卖场", "竞争性磋商", "竞争性谈判", "询价", "邀请招标"]

mask = ~df_filtered["采购方式"].isin(methodlist)

df_filtered.loc[mask, "采购方式"] = df_filtered.loc[mask, "采购方式"].apply(
    lambda x: clean_method(x, methodlist)
)


In [234]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539860 entries, 0 to 539859
Data columns (total 27 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   合同编号         539792 non-null  object 
 1   项目编号         536972 non-null  object 
 2   项目名称         539858 non-null  object 
 3   采购人甲方        539857 non-null  object 
 4   采购人地址        539860 non-null  object 
 5   采购人联系方式      539836 non-null  object 
 6   供应商乙方        539859 non-null  object 
 7   供应商地址        539851 non-null  object 
 8   供应商联系方式      539831 non-null  object 
 9   主要标的名称       539853 non-null  object 
 10  规格型号或服务要求    539818 non-null  object 
 11  主要标的数量       539853 non-null  object 
 12  主要标的单价       539852 non-null  object 
 13  合同金额万元       539860 non-null  float64
 14  履约期限地点等简要信息  537838 non-null  object 
 15  采购方式         439785 non-null  object 
 16  合同签订日期       539813 non-null  object 
 17  合同公告日期       539860 non-null  object 
 18  年份           539860 non-

In [235]:
df_filtered.to_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_primary.csv", index=False, encoding="utf-8-sig")

df_filtered_2cat = df_filtered[df_filtered["cat"].isin(["服务", "货物"])]
df_filtered_2cat.to_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_2cat.csv", index=False, encoding="utf-8-sig")

## policy threshold

In [102]:
import pandas as pd


df = pd.read_excel(
    "/Users/yxy/UChi/Summer2025/Procurement/raw/China_Procurement_Thresholds_2014_2024.xlsx"
)

df = df[["prov", "year", "cat", "threshold"]]

years = list(range(2020, 2025))
records = []

for prov, g1 in df.groupby("prov"):
    for cat, g2 in g1.groupby("cat"):
        g2 = g2.sort_values("year")
        min_year, max_year = g2["year"].min(), g2["year"].max()
        min_thr = g2.loc[g2["year"] == min_year, "threshold"].iloc[0]
        max_thr = g2.loc[g2["year"] == max_year, "threshold"].iloc[0]
        vals = []
        for y in years:
            if y <= min_year:
                thr = min_thr
            elif y >= max_year:
                thr = max_thr
            else:
                thr = g2[g2["year"] <= y].iloc[-1]["threshold"]
            vals.append(thr)
            records.append([prov, cat, y, thr, None]) 
        change_flag = int(len(set(vals)) > 1)

        for i in range(len(years)):
            records[-len(years) + i][4] = change_flag

threshold = pd.DataFrame(records, columns=["prov", "cat", "year", "threshold", "changed"])

threshold.to_csv(
    "/Users/yxy/UChi/Summer2025/Procurement/dta/threshold.csv",
    index=False,
    encoding="utf-8-sig"
)


In [103]:
threshold['changed'].sum()

np.int64(60)

In [202]:
df = pd.read_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_2cat.csv", low_memory=False)
df['主要标的名称'].value_counts().head(20)

主要标的名称
物业管理服务       22234
详见附件          4731
详见合同          3950
其他服务          2894
物业服务          1928
无无            1501
安全服务          1438
触控一体机         1238
其他不另分类的物品     1123
台式计算机         1103
保安服务          1094
无              978
详见合同文本。        918
空调机            639
电梯             631
服务             629
LED显示屏         623
详见合同附件         618
见附件            603
审计服务           502
Name: count, dtype: int64